# <center> LangChain基础入门

前言

开源大模型系列的课程至今已经详细介绍了围绕ChatGLM3-6b模型的部署、微调和简单调用方法。私有化部署大模型是开发大模型应用中最基础的一步，但如果像熟练掌握基于大模型的上层应用开发还有非常长的路要走。现如今的大模型开发总体上分为两类，一种是以OpenAI为代表的闭源的模型在线API应用开发，而另一类是先本地私有化部署开源大模型，再借助开发框架构建大模型应用。值得注意的是，目前主流的大模型应用开发框架也支持在线模型API的调用，这正是我们之前一直强调学习掌握开源或在线API调用技巧的原因，因为这些方法和技巧直接对应于应用开发的最佳实践。

相较于闭源模型已经封装好的功能应用只需要调用API并构建合理即可实现应用，在开源大模型的技术领域下想要实现对应的功能，重点在于如何调整tools与Schema表示，因此在整个开发过程中需要花费大量的时间而并非直接与模型调用相关，而是在于深入理解和应用这些底层原理和技术细节。也就是说在构建应用的过程中，虽然LLM是核心但是实际针对LLM编写的代码量其实较小，而在这部分有限的代码中，Prompt提示工程往往又会占据主要的工作量。这进一步引出了一个关键问题，即：除了大模型本身其他环节怎样才能实现高效合作，以及这些环节中的代码开发和工作量应该如何分配和执行，将会在很大程度上影响应用的开发效率和最终的用户体验。



目录 
一 整体介绍<br>
&emsp;1 Langchain的定位<br>
&emsp;2 LangChain的生态<br>
&emsp;3 LangChain的构成<br>
二 LangChian全面部署方法<br>
&emsp;1 使用包版本管理工具安装<br>
&emsp;2 源码安装<br>
三 LangChian调用方法<br>
&emsp;1 LangChain表达式语言--ICEL<br>
&emsp;2 以ChatGLM3为基座模型实现启动<br>
四 大模型显存计算工具<br>
&emsp; Model Memory Calculator

## 1. 整体介绍

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920152033456.png" width=80%></div>

- Langchain的定位

LangChain给自身的定位是：用于开发由大语言模型支持的应用程序的框架。它的做法是：通过提供标准化且丰富的模块抽象，构建大语言模型的输入输入规范，利用其核心概念chains，灵活地连接整个应用开发流程。而针对每个功能模块，都源于对大模型领域的深入理解和实践经验，开发者提供出来的标准化流程和解决方案的抽象，再通过灵活的模块化组合，才有了目前这样一款在大模型应用开发领域内被普遍高度认可的通用框架。

- 该产品出现的原因

首先，大模型发展的态势仍旧是百花齐放，而每一家公司的模型都有自己的调用格式和端口，这为学习新的模型或实现大模型替换时带来诸多不便。例如：从LLM兴起之初就开始的开源模型Claude系列模型的调用方法和Open AI的GPT系列就不兼容，甚至OpenAI自家的API调用方式在上半年就发生过变化。如果每出一个新模型就要花费大量的实践学习特定的规范在进行应用探索只会消耗开发人员的时间和精力。

另外，每个大模型都有自己的局限性，例如训练的知识库具有截至日期，询问之后的消息如果不借助其他工具或定义Prompt就得不到理想的结果。而且大语言模型本质上只有文字输入输出功能，如何优雅的实现数据连接和处理是应用开发时需要重点考虑的问题。尽管通过微调等方式可以增强模型的原生能力，但这样也会增加成本和研究投入，可能背离一开始的开发初衷。

从本质上分析，LangChain还是依然采用从大模型自身出发的策略，通过开发人员在实践过程中对大模型能力的深入理解及其在不同场景下的涌现潜力，使用模块化的方式进行高级抽象，设计出统一接口以适配各种大模型，从而简化了与LLM的交互成本。

通过LangChain，开发者不需要深入了解每个模型的细节，只需要使用LangChain提供的接口即可完成复杂的任务。此外LangChain的组件化设计允许开发者根据需要选择不同的工具和功能。这些组件包括提示词Prompt管理、语言模型接口、输出解析器等，进而使得开发者灵活构建和拓展应用程序。LangChain支持数据连接和处理功能，可以轻松连接各种数据源（eg：PDF、CSV、base data）并将数据转换为适合LLM处理的格式。此外LangChain可以实现高效的信息检索，通过将文档切分成较小的部分并存贮在向量数据库中，当用户提出问题时，系统会将问题转换为向量形式，并于数据库中的数据向量进行对比，从而找到最相关的答案。除了基本的端口优化功能，LangChain还可以通过将多个组件结合起来自动化地完成复杂任务。

LangChain已经被广泛的应用于企业环境中，同时作为一个开源项目，LangChain拥有活跃的社区支持。开发者可以免费的使用LangChain，并从社区中获得帮助和资源。从LangChain的GitHub版本迭代历史上看，从2022年10月开源以来已经经历了3个大版本百余个小版本的更新，如今仍然以频率的更新在加速项目的功能上线，基本上每出一个新的大模型都会在一天左右的时间出现适配的LangChain端口。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920110917678.png" width=60%></div>

- LamgChain的生态

如今Lanchain已经形成了一套丰富的框架体系，包括LangChain、LangGraph、LangGraph Cloud 和 LangSmith ，它们都是与语言模型（LLM）及其工具生态相关的框架或服务，下面是它们的关系和主要功能区别：

**LangChain**:

- **核心功能**: LangChain 是一个 Python 和 JavaScript 框架，专注于将语言模型（LLM）集成到各种应用程序中。它提供了工具链和库，用于构建复杂的基于 LLM 的应用，如问答系统、聊天机器人、自动化工作流等。LangChain 强调的是将 LLM 与外部工具（如数据库、API）连接，从而实现更强大的功能。
- **应用场景**: 主要用于创建和管理与 LLM 交互的链条（chains），例如多个语言模型任务的执行顺序、逻辑控制等。

**LangGraph**:

- **核心功能**: LangGraph 是一个将 LLM 和图数据库（graph database）结合的工具，支持在语义层次上通过语言模型构建和查询复杂的知识图谱。通过 LangGraph，可以在图结构中执行自然语言查询，并通过 LLM 来进行数据增强和推理。
- **应用场景**: 主要用于知识管理、复杂数据查询和推理等场景。

**LangGraph Cloud**:

- **核心功能**: LangGraph Cloud 是基于云的 LangGraph 服务，提供托管的 LangGraph 功能。用户可以通过云端服务直接使用图数据库和 LLM 集成功能，而不必自行部署和维护基础设施。
- **应用场景**: 提供了一种更加简便的方式来使用 LangGraph，特别适合需要大规模或云端部署的项目。

**LangSmith**:

- **核心功能**: LangSmith 是 LangChain 开发者最近推出的一个工具，主要用于调试和监控 LLM 应用程序的性能。它提供了实验追踪、性能分析和调试工具，使开发者能够更好地理解和优化 LLM 驱动的应用程序的行为。
- **应用场景**: 用于开发和部署阶段的性能跟踪和优化，帮助开发者提升应用程序的效率和准确性。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240919183418501.png" width=60%></div>

简单来说这几个工具有以下的关系：
- LangChain 是基础框架，用于构建基于 LLM 的链条应用。
- LangGraph 是一种特定用途的工具，将 LLM 与图数据库集成。构建具有流式处理和人机回圈支持的状态代理。
- LangGraph Cloud 提供 LangGraph 的云托管版本，简化了部署和使用。
- LangSmith 则是为 LangChain 提供性能分析、调试支持的工具。检查、监控和评估应用程序，以便可以高效地不断优化和部署。

这几个工具构成了一个完整的生态系统，帮助开发者在不同的应用场景下构建、管理和优化基于 LLM 的应用。

- *LangChain的构成*

&emsp;&emsp;从本质上分析，LangChain还是依然采用从大模型自身出发的策略，通过开发人员在实践过程中对大模型能力的深入理解及其在不同场景下的涌现潜力，使用模块化的方式进行高级抽象，设计出统一接口以适配各种大模型。到目前为止，LangChain抽象出最重要的核心模块如下：

1. Model I/O ：标准化各个大模型的输入和输出，包含输入模版，提示优化，模型本身和格式化输出；
2. Retrieval ：检索外部数据，然后在执行生成步骤时将其传递到 LLM，包括文档加载、切割、Embedding等；
3. Chains ：链条，LangChain框架中最重要的模块，链接多个模块协同构建应用，是实际运作很多功能的高级抽象；
4. Memory ： 记忆模块，以各种方式构建历史信息，维护有关实体及其关系的信息；
5. Agents ： Agent允许 LLM 自主决定如何完成任务。Agent决定要采取哪些操作，然后执行该操作并观察结果，并重复直到任务完成。；
6. Callbacks ：回调系统，允许连接到 LLM 应用程序的各个阶段。用于日志记录、监控、流传输和其他任务；

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920151136100.png" width=70%></div>


通过这些组件的协同工作，LangChain能够有效应对大型语言模型在实际应用中面临的两大挑战：一是模型接口的复杂性，二是输入长度受到模块设计的限制。LangChain的设计旨在简化这些模型的使用，提高其灵活性和扩展性，从而更好地服务于各种应用场景。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202403071044331.png" width=80%></div>

&emsp;&emsp;从上图中可以看到，LangChain框架涵盖了模型输入输出的标准化、外部工具接入的规范、上下文记忆功能，以及对数据库、SQL、CSV等多种数据源的连接标准。通过核心的"Chain"高级抽象，定义了不同形式下标准的链接方法，这就能够允许开发者根据实际的应用需求和数据流向快速构建出一套完整的应用程序。这个过程类似于搭建积木，可以灵活适应不同的任务需求。

&emsp;&emsp;也正因为如此，LangChain中涉及的概念和模块化是非常多的，每个模块都有其独特的使用场景和使用方法，那么如何去搭这个“积木”，就需要我们对其每个核心模块都要有一个比较清楚的认知。本次课先为大家带来一个基础的概念认知，在后续的内容里会逐渐丰富完善Langchian部分的各个基础模块，彼时大家将会有一个清晰的认知并可以结合自身的实际项目内容选择合适的构造模块搭建自己的工作流。

- 产品定位

作为一个大模型应用开发框架，Langchian具有以下优势：
1. 提示词工程构建功能，帮助开发者设计和优化输入提示，以提高模型的响应质量和准确性。这些工具支持定制化提示模板，使得开发者能够根据特定任务或应用需求调整模型的输入。
2. 输出解释器，这些解释器能够对模型的输出进行解析和后处理，提取关键信息，并将其转换成更易于理解和操作的格式。这对于提高应用的用户体验和实现复杂的数据处理流程至关重要。
3. RAG数据增强工具，这些工具可以通过检索相关信息来增强模型的生成能力。这意味着模型不仅依赖于训练数据，还能够动态地利用最新的信息来提升输出的相关性和准确性。
4. 灵活的链式结构：LangChain 支持构建复杂的链式架构，允许开发者将不同的模型和组件链接在一起，形成一个处理流程。这种模块化的方法使得应用开发更加灵活，便于扩展和维护。

同样的，作为一个高度抽象的功能的框架为开发带来一些缺陷：

1. 部分接口的抽象级别较高，这可能使得初学者在理解源代码方面遇到困难，从而增加了对源码解读的需求。
2. 框架中复杂的嵌套结构可能对初学者构成挑战，使他们感到迷惑。
3. 在Agent场景中，使用LangChain可能会带来额外的性能开销，因为Agent在执行过程中需要频繁调用LLM API。
4. 目前，许多提示词工程尚未充分支持中文，这限制了框架在中文语境下的应用潜力。

## 2. 安装方法

&emsp;&emsp;LangChain的安装过程非常简单，可以通过常用的Python包管理工具，如pip或conda，直接进行安装。稍复杂一点的还可以通过源码进行安装。但有一点大家一定要明确：LangChain的真正价值在于它能够与多种模型提供商、数据存储解决方案等进行集成。默认情况下，使用上述两种安装方式中的任意一种来进行LangChain安装后，安装的仅仅是LangChain的默认功能，并不包括这些集成所需的额外依赖项。也就是说，如果我们想要使用特定的集成功能，还需要额外安装这些特定的依赖。以调用OpenAI的API为例，我们首先需要通过运行命令`pip install langchain-openai`安装OpenAI的合作伙伴包，安装此依赖包后，LangChain才能够与OpenAI的API进行交互。后续我们在使用相关功能的时候，会提供额外的说明。

> LangChain安装官方说明文档：https://python.langchain.com/docs/get_started/installation

### 2.1 使用包版本管理工具安装

&emsp;&emsp;**LangChain可以使用pip 或者 conda直接安装，适用于仅使用的场景，即不需要了解其源码构建过程。**这种安装方法十分简洁明了，只需执行一条命令，就可以在当前的虚拟环境中迅速完成LangChain的安装。具体操作如下：

In [1]:
! pip install langchain

&emsp;&emsp;**注：如果是在Jupyter lab操作，需要重启当前的Jupyter Lab使配置生效。**

&emsp;&emsp;重启完当前的Jupyter Lab后，验证LangChain的安装情况，执行命令如下：

In [ ]:
import langchain

print(langchain.__version__)

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240919184536317.png" width=80%></div>

&emsp;&emsp;如果能正常输出LangChain的版本，说明在当前环境下的安装成功。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920103919004.png" width=100%></div>

In [ ]:
pip install -qU langchain langchain-community

&emsp;&emsp;在下载安装的时候安装包会影响整个版面的输出，也可以在pip install后面加上-qU来实现静默快速安装/升级。

-q：这是 --quiet 的简写形式，表示减少输出的日志信息，使得安装过程更加安静，通常用于自动化脚本中以避免过多的输出干扰。

-U：这是 --upgrade 的简写形式，表示升级包。如果指定的包已经安装，这个选项会将包升级到最新版本。如果没有安装，则会安装最新版本的包

### 2.2 源码安装

&emsp;&emsp;除了通过pip安装外，还有一种通过源码安装的方法。这需要使用git拉取远程仓库，然后进入项目文件夹并执行`pip install -e .`命令。在使用` pip install -e . `时，`.` 表示当前目录下的项目文件会被以可编辑模式安装。这种方法不仅会安装必要的依赖，同时也将程序的源代码保存在本地。通常来说，除非大家打算作为LangChain的协作开发者参与到项目的功能更新中，否则没必要采用这种安装方式，直接使用pip安装更为简便。

&emsp;&emsp;**对于课程学习而言，我们推荐采用源码安装方式，这将非常有助于在后续的LangChain功能探索中，通过源码分析深入理解框架的构建原理和详细机制。**

&emsp;&emsp;接下来我们以Ubuntu操作系统为例，介绍源码安装LangChain的详细步骤：

- **Step 1. 安装Anaconda**

> 注：如果部署环境是租赁的云服务器，Anaconda环境已经预设好，并不需要我们再次手动配置。

&emsp;&emsp;首先确认配置好Anaconda环境，可以通过如下命令验证当前环境的Conda：
```bash
conda --version
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920151449524.png" width=100%></div>

&emsp;&emsp;如果可以正常输出Conda的版本，则说明Anaconda安装成功。

- **Step 2. 使用Conda创建LangChain的Python虚拟环境**

&emsp;&emsp;安装好Anaconda后，我们需要借助Conda包版本工具，为LangChain项目创建一个新的Python虚拟运行环境，执行代码如下：
```bash
 conda create --name langchain python==3.11
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202403061404163.png" width=100%></div>

&emsp;&emsp;如上所示，新创建了一个名为langchain的Python虚拟环境，其Python版本为3.11。创建完成后，通过如下命令进入该虚拟环境，执行后续的操作：
```bash
    conda activate langchain
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202403061418251.png" width=100%></div>

- **Step 3. 下载LangChain的项目文件**

&emsp;&emsp;进入LangChain的官方Github，地址：https://github.com/langchain-ai/langchain ， 在 GitHub 上将项目文件下载到有两种方式：克隆 (Clone) 和 下载 ZIP 压缩包。推荐使用克隆 (Clone)的方式。我们首先在GitHub上找到其仓库的URL。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202403061418252.png" width=100%></div>

&emsp;&emsp;在执行命令之前，需要先安装git软件包，执行命令如下：
```bash
sudo apt install git
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202403011057828.png" width=100%></div>

&emsp;&emsp;执行克隆命令，将LangChain Github上的项目文件下载至本地的当前路径下，如下：
```bash
    git clone https://github.com/langchain-ai/langchain.git
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920151812618.png" width=100%></div>

- **Step 4. 升级pip版本**

&emsp;&emsp;建议在执行项目的依赖安装之前升级 pip 的版本，如果使用的是旧版本的 pip，可能无法安装一些最新的包，或者可能无法正确解析依赖关系。升级 pip 很简单，只需要运行命令如下命令：

```bash
python -m pip install --upgrade pip
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202403061418254.png" width=100%></div>

- **Step 5. 源码安装项目依赖**

&emsp;&emsp;不同于我们之前一直使用的`pip install -r requirements.txt`方式，这种方法用于批量安装多个依赖包，是在部署项目或确保开发环境与其他开发者/环境一致时的常用方式。而对于LangChain，我们需要使用`pip install -e `的方式，以可编辑模式安装包。这种方式主要用于开发过程中。当以可编辑模式安装一个包时，依赖包会被直接从源代码所在位置安装，而不是复制到Python的site-packages目录，是开发模式下用于安装并实时反映对本地包更改的方法。需要执行的步骤如下：

```bash
cd langchain/libs/langchain/

pip install -e .
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920151858267.png" width=100%></div>

&emsp;&emsp;如在安装过程未发生任何报错，则说明安装成功。

- pip install -e 是一个用于安装 Python 包的命令，其中 -e 或 --editable 是一个选项，表示以“可编辑”模式安装包。

&emsp;&emsp;当使用 pip install -e 命令时，它会将包安装在当前目录，而不是将其复制到 Python 的 site-packages 目录中。这意味着你对包的代码所做的任何更改都会立即反映在安装的包中，无需重新安装。这在开发过程中非常有用，因为它允许开发者在不中断开发流程的情况下进行迭代和测试。

## 3. LangChain表达式语言-ICEL

官方的网址链接如下：https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

`LangChain Expression Language`是链接LangChain组件的声明性方式。这个语言模式是为了将LLM在无需修改的条件下投入生产，通过统一的语法模式可以有效的调用模型。使用 LCEL 构建的流媒体解决方案具备以下特点：


- 卓越的流媒体性能：LCEL 确保在构建链时能够实现最快的`token`获取时间，即从链的开始到第一块输出生成的时间。对于某些链，这表示能够以与 LLM 提供商相同的速率，将`token`从 LLM 直接流式传输到输出解析器，并以增量方式返回解析后的数据块。

- 异步处理能力：LCEL 支持同步和异步 API 调用，使得无论是在 Jupyter 笔记本中进行原型设计，还是在 LangServe 服务器中进行生产环境部署，都能使用相同的代码。这种设计不仅提升了性能，还允许服务器高效处理多个并发请求。

- 优化的并行处理：LCEL 能够自动识别并行执行的步骤，无论是在同步还是异步接口中，都能自动执行以减少延迟。

- 重试与回退机制：LCEL 允许为链的任何部分配置重试和回退策略，增强了链在大规模应用时的可靠性。目前，LCEL 正在开发流式处理的重试/回退支持，以确保在不增加延迟的情况下提高可靠性。

- 中间结果访问：在处理复杂链时，能够访问中间步骤的结果对于用户了解进度或进行调试非常有用。LCEL 提供了流式传输中间结果的功能，且在每个 LangServe 服务器上均可用。

- 输入输出架构：LCEL 为每个链提供基于 Pydantic 和 JSONSchema 的输入输出架构，这有助于验证数据的准确性，并是 LangServe 的核心组成部分。

- LangSmith 集成：随着链的复杂性增加，跟踪每一步的执行变得至关重要。LCEL 自动将所有步骤记录到 LangSmith 中，以提供强大的可观察性和调试能力。

所定义的输入和输出类型如下：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920120015425.png" width=60%></div>

## Model I/O调用实例：以ChatGLM3-6B为例实现远程调用

&emsp;&emsp;LangChain 提供了一套与任何大语言模型进行交互的标准构建模块。所以需要明确的一点是：**虽然 LLMs 是 LangChain 的核心元素，但 LangChain 本身不提供 LLMs，它仅仅是为多种不同的 LLMs 进行交互提供了一个统一的接口。**简单理解：以OpenAI的GPT系列模型为例，如果我们想通过 LangChain 接入 ChatGLM 的 chat 模型，我们需要在LangChain框架下先定义相关的类和方法来规定如何与模型进行交互，包括数据的输入和输出格式以及如何连接到模型本身。然后按照 OpenAI GPT 模型的接口规范来集成这些功能。通过这种方式，LangChain 充当一个桥梁，使我们能够按照统一的标准来接入和使用多种不同的大语言模型。

&emsp;&emsp;**在LangChain的Model I/O模块中，集成了两类模型，分别是LLMs和聊天模型，这两种模型在输入和输出的数据类型上有所区别。**

首先在终端的Anaconda prompt中下载langchain-community包

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920161640624.png" width=50%></div>

In [ ]:
pip install -qU langchain langchain-community

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920162700255.png" width=80%></div>

建立一个python文件，复制以下代码以下载对应的向量库模型。

这样的代码模式通常用于需要在本地运行或使用特定版本模型的场景下，特别是希望确保模型是从远程仓库中获取的最新或指定版本。

from modelscope.hub.snapshot_download import snapshot_download


snapshot_download('AI-ModelScope/bge-large-zh-v1.5', revision='master')

接下来可以在Jupyter Lab文件内执行以下代码，导入langchain封装好的库实现功能调用。

In [2]:
from langchain.chains import LLMChain
from langchain_community.llms.chatglm3 import ChatGLM3
from langchain_core.messages import AIMessage
from langchain_core.prompts import PromptTemplate

- `langchain_core`：基本抽象和 LangChain 表达式语言。
- `langchain_community`：第三方集成。
  - 一些集成已进一步拆分为仅依赖 `langchain-core` 的伙伴包。示例包括 `langchain_openai` 和 `langchain_anthropic`。
- `langchain`：构成应用程序认知架构的链、代理和检索策略。
- `LangGraph`：一个库，用于使用 LLM 构建健壮且有状态的多参与者应用程序，方法是将步骤建模为图形中的边和节点。与 LangChain 无缝集成，但可以在没有 LangChain 的情况下使用。

In [2]:
template = """{question}"""
prompt = PromptTemplate.from_template(template)

- 第一行定义了一个模板字符串 template，其中包含了一个占位符 {question}。{question} 是一个动态变量，它将被替换为实际的内容（如具体的问题）在后续步骤中使用。
- PromptTemplate.from_template(template)：
这是一个类方法，它允许你基于一个简单的模板字符串创建一个 PromptTemplate 实例。
通过这种方式，PromptTemplate 可以将占位符（如 {question}）与实际输入（如用户的问题）相结合，生成一个完整的提示。

接下来进行服务器和主机终端的互联，在命令行输入`ipconfig`指令可以看到对应机器的网络适配器，其中IPv4地址是局域网中唯一标识。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920162913014.png" width=60%></div>

在服务器终端打开api_server.py文件修改并保存IP地址和端口号并保存退出。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920162112354.png" width=60%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920161232034.png" width=90%></div>

调整好网络端口后，在服务器终端启动api_server.py文件即可实现服务器和局域网内终端的互联。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920160959805.png" width=100%></div>

- 通过在命令行输入ipconfig可以看到对应机器的网络适配器，其中IPv4地址是局域网中唯一标识。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920162913014.png" width=60%></div>

完成局域网互联后，继续通过Langchian进行Prompt和大模型参数的赋值，通过AIMessage的方式可以建立多个prompt以提示是大模型的输出内容，在llm参数中可以设置最大输出token数、输入信息以及输出的灵活度。

In [7]:
endpoint_url = "http://192.168.110.133:8000/v1/chat/completions"

messages = [
    AIMessage(content="我将从美国到中国来旅游，出行前希望了解中国的城市"),
    AIMessage(content="欢迎问我任何问题。"),
]

llm = ChatGLM3(
    endpoint_url=endpoint_url,
    max_tokens=80000,
    prefix_messages=messages,
    top_p=0.9,
)

In [8]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "北京和上海两座城市有什么不同？"

llm_chain.run(question)

'北京和上海是中国两个不同的城市,它们在很多方面都有所不同。\n\n北京是中国的首都,也是历史悠久的城市之一。它有着丰富的历史文化遗产,如故宫、天安门、颐和园等,这些都是值得游览的景点。北京也是一个政治、文化和教育中心,有很多政府机构和大学。\n\n上海则是一个现代化的城市,是中国经济最发达的城市之一。它有很多高楼大厦,是商业和金融中心。上海也有许多文化景点,如外滩、城隍庙等,这些景点都值得游览。\n\n北京和上海都是中国的重要城市,每个城市都有自己独特的魅力,您可以根据自己的兴趣选择适合自己的旅游方式。'

---

一般ChatGLM3调用方法

在python环境中继续输入以下代码调用 ChatGLM3-6B 模型来生成对话：
```python
from modelscope import AutoTokenizer, AutoModel, snapshot_download
model_dir = snapshot_download("ZhipuAI/chatglm3-6b", revision = "v1.0.0")
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModel.from_pretrained(model_dir, trust_remote_code=True).half().cuda()
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)
```


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/18.png" width=100%></div>

---

&emsp;&emsp;相较于一般的启动方法，langchain看上去并没节省许多代码，但是它通过直接封装调用方式使得一套启动语法可启动任意LLM，只需替换想要执行的LLM名称即可，这在宏观操作上实际会带来极大的便利。

## 大模型显存需求计算工具

网址连接：https://huggingface.co/spaces/hf-accelerate/model-memory-usage

在进行大模型、微调、训练的时候，相信大家尝尝会遇到这样的苦恼，即：不知道自己的机器是否可以胜任对应的大模型推理/训练任务，或者不知道想要训练对应目标的大模型需要多大的显卡配置等，今天为大家推荐的是一款用来解决这一问题的实用小程序，这个工具将帮助您计算在 Hugging Face Hub上托管的模型进行训练和大型模型推理所需的vRAM（虚拟随机存取存储器）数量。模型所需的最小推荐vRAM以“最大层”的大小表示，而模型训练大致需要其大小的4倍（对于Adam优化器）。这些计算的准确性最多在几个百分点以内，例如bert-base-cased模型为413.68 MB，而计算器估计为413.18 MB。在执行推理时，预计需要额外增加高达20%，这是由EleutherAI发现的。将来会进行更多测试，以获得每个模型更准确的基准。目前，这个工具支持所有使用transformers和timm的托管模型。要使用这个工具，请输入您想要计算内存使用量的模型的URL或模型名称，选择它源自哪个框架（“auto”将尝试从模型元数据中检测），以及您想要使用的精度。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920165928005.png" width=60%></div>

使用方式非常简单：只需输入在hugging face上托管的对应模型名称，推理方式，推理精度信息后选择计算即可。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240920171123377.png" width=60%></div>

如上图的计算结果，显示了模型基础占用的显存是11.62G，在使用 Adam 优化器进行训练时，需要再额外的 11.63 GB 显存来存储梯度信息，反向传播过程中占用了 23.26 GB 显存，因此显存的峰值达到了 46.52 GB。
